In [1]:
# importing packages
import pandas as pd
import numpy as np

In [2]:
data1 = pd.ExcelFile('Resources/indiandataset.xls')
data1 = data1.parse(0)
data1.head()

,label,msg
0,ham,"Dear Customer, +916300623587 is now available ..."
1,ham,"Dear Customer, You have a missed call from +91..."
2,spam,"Join Hike to get Rs 40. Earn upto Rs. 10,000 b..."
3,spam,Just sent you some money and invited you to Hi...
4,spam,Just sent you some money and invited you to Hi...


In [3]:
data2 = pd.read_csv('Resources/spam.csv', encoding = 'latin-1')
data2 = data2.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
data2.head()

,label,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
print(len(data1))
print(len(data2))

data = pd.concat([data1, data2])
print(len(data))
data.head()

4596
5572
10168


,label,msg
0,ham,"Dear Customer, +916300623587 is now available ..."
1,ham,"Dear Customer, You have a missed call from +91..."
2,spam,"Join Hike to get Rs 40. Earn upto Rs. 10,000 b..."
3,spam,Just sent you some money and invited you to Hi...
4,spam,Just sent you some money and invited you to Hi...


In [5]:
# encoding Data
data['label'] = pd.factorize(data['label'])[0]

In [6]:
# module for removing unwanted words
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

# for stemming words
from nltk.stem.porter import PorterStemmer
temp = []

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manohar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
for row in data.itertuples():
    # to keep a - z letters and 0 - 9
    rev = str(row[2])
    rev = rev.split()
    rev = [word.lower() for word in rev]
    ps = PorterStemmer()
    rev = [ps.stem(word) for word in rev if not word in set(stopwords.words("english"))]
    rev = " ".join(rev)
    temp.append(str(rev))

In [8]:
data['msg'] = temp

In [7]:
# splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data['msg'], data['label'], test_size=0.25, random_state=0)

In [8]:
# just for testing
for row in X_train:
    # to keep a - z letters and 0 - 9
    rev = str(row)
    rev = rev.split()
    rev = [word.lower() for word in rev]
    ps = PorterStemmer()
    rev = [ps.stem(word) for word in rev if not word in set(stopwords.words("english"))]
    rev = " ".join(rev)
    temp.append(str(rev))
X_train = temp

In [9]:
# creating bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 550)
cv.fit(X_train)
X_train = cv.transform(X_train)

In [26]:
# Training classifier
from sklearn import svm
classifier = svm.SVC()
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [27]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2002    6]
 [ 285  249]]


stacking

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from sklearn.linear_model import LogisticRegression

classifier = make_pipeline(StackingEstimator(estimator=GaussianNB()), LogisticRegression())
classifier.fit(X_train.toarray(), y_train)

Pipeline(memory=None,
     steps=[('stackingestimator', StackingEstimator(estimator=GaussianNB(priors=None))), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [12]:
#y_pred = classifier.predict(X_test.toarray())

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2004   31]
 [  73  434]]


In [16]:
from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(y_test, y_pred))

0.8690105624555238


# wrongly predicted

In [17]:
import csv
file = open('output/wrongly_predicted.csv', 'w')
wr_file = csv.writer(file)
wr_file.writerow(["label","msg"])

11

In [11]:
temp = []
# just for testing
for row in X_test:
    # to keep a - z letters and 0 - 9
    rev = str(row)
    rev = rev.split()
    rev = [word.lower() for word in rev]
    ps = PorterStemmer()
    rev = [ps.stem(word) for word in rev if not word in set(stopwords.words("english"))]
    rev = " ".join(rev)
    temp.append(str(rev))

temp = cv.transform(temp)
y_pred = classifier.predict(temp.toarray())

In [18]:
j = 0
for i in range(len(y_test)):    
    if not y_test.iloc[i] == y_pred[i]:
        j = j + 1
        if y_test.iloc[i] == 0:
            te = 'ham'
        else:
            te = 'spam'
        wr_file.writerow([te,X_test.iloc[i]])
file.close()

In [16]:
print(y_test.iloc[0])

1
